# Section 3: Homework Exercises

This material provides some hands-on experience using the methods learned from the third day's material.

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import scipy.stats as st
import pymc as pm
import arviz as az

## Exercise: Effects of coaching on SAT scores

This example was taken from Gelman *et al.* (2013):

> A study was performed for the Educational Testing Service to analyze the effects of special coaching programs on test scores. Separate randomized experiments were performed to estimate the effects of coaching programs for the SAT-V (Scholastic Aptitude Test- Verbal) in each of eight high schools. The outcome variable in each study was the score on a special administration of the SAT-V, a standardized multiple choice test administered by the Educational Testing Service and used to help colleges make admissions decisions; the scores can vary between 200 and 800, with mean about 500 and standard deviation about 100. The SAT examinations are designed to be resistant to short-term efforts directed specifically toward improving performance on the test; instead they are designed to reflect knowledge acquired and abilities developed over many years of education. Nevertheless, each of the eight schools in this study considered its short-term coaching program to be successful at increasing SAT scores. Also, there was no prior reason to believe that any of the eight programs was more effective than any other or that some were more similar in effect to each other than to any other.

You are given the estimated coaching effects (`d`) and their sampling variances (`s`). The estimates were obtained by independent experiments, with relatively large sample sizes (over thirty students in each school), so you can assume that they have approximately normal sampling distributions with known variances variances.

Here are the data:

In [ ]:
J = 8
d = np.array([28.,  8., -3.,  7., -1.,  1., 18., 12.])
s = np.array([15., 10., 16., 11.,  9., 11., 10., 18.])

Construct an appropriate model for estimating whether coaching effects are positive, using a **centered parameterization**, and then compare the diagnostics for this model to that from an **uncentered parameterization**.

Finally, perform goodness-of-fit diagnostics on the better model.

In [ ]:
schools = np.array(
    [
        "Choate",
        "Deerfield",
        "Phillips Andover",
        "Phillips Exeter",
        "Hotchkiss",
        "Lawrenceville",
        "St. Paul's",
        "Mt. Hermon",
    ]
)

with pm.Model(coords={'school': schools}) as schools_centered:
    
    mu = pm.Normal("mu", 0, sigma=1e6)
    tau = pm.HalfCauchy("tau", 5)

    theta = pm.Normal("theta", mu, sigma=tau, dims='school')

    obs = pm.Normal("obs", theta, sigma=s, observed=d)

    trace_centered = pm.sample()

In [ ]:
with pm.Model(coords={'school': schools}) as schools_noncentered:
    
    mu = pm.Normal("mu", 0, sigma=1e6)
    tau = pm.HalfCauchy("tau", 5)

    z = pm.Normal("z", 0, sigma=1, dims='school')
    theta = mu + z * tau

    obs = pm.Normal("obs", theta, sigma=s, observed=d)

    trace_noncentered = pm.sample()

In [ ]:
with schools_centered:
    pm.compute_log_likelihood(trace_centered)

In [ ]:
with schools_noncentered:
    pm.compute_log_likelihood(trace_noncentered)

In [ ]:
az.compare({'centered': trace_centered, 'non-centered': trace_noncentered}, ic='loo')

In [ ]:
with schools_centered:
    pm.sample_posterior_predictive(trace_centered, extend_inferencedata=True)

In [ ]:
az.plot_ppc(trace_centered)

## Exercise: Car Price Prediction

We will use a small subset of [this Kaggle dataset](https://www.kaggle.com/datasets/nehalbirla/vehicle-dataset-from-cardekho). This data is amenable to both multiple linear regression and hierarchical regression, you can start with the former and move on to the latter. The data has the selling price of multiple used cars, along with some other features such as year of production, kilometers driven, fuel type, car brand and model, etc.

The full dataset has a lot of information, but we chose to crop it down to a simpler form that is easier for you to work with. Our reduced dataset has the following features:

- Selling price in thousands of dollars
- Natural log of the kilometers driven
- Year of production
- Number of seats
- Brand name
- Car name
- Full name of the car (with special edition modifiers)
- Kilometers per liter of fuel
- Engine CC

The goal of this exercise is to be able to **predict the selling price based on the rest of the features using linear regressions**. We propose that you do the following steps to do so:

1. Do an explorative data analysis. Visualize the distribution of selling prices and how it relates to the other features. Look at how the distribution changes across brands. Try to reason whether it's appropriate to transform the observations in some way to ellicit a linear relationship with some features in the dataset.
2. Assuming the observations are normally distributed, write a series of linear regression models to fit the dataset
    a. Try a model with a single intercept parameter and do prior predictive analysis to determine appropriate priors.
    b. Add linear terms that relate the model with the numerical features in the data. Try to reason about adequate priors for the slopes, and whether it helps to standardize the features or not.
    c. Run inference on both models and explain what happens to the observational noise parameter.
3. Add a hierarchical structure that depends on the car brand. Do prior predictive simulations to choose appropriate priors, and run inference. The hierarchical structure could be on the intercept term, the slopes or both.
4. (HARD) Add a nested hierarchy term to the model's intercept. The first level of the hierarchy is the brand, the second layer is the car name. Explore what happens if you use the centered or the non-centered parametrization in the second layer of the hierarchy.
5. Compare all of the above models using `arviz.compare`. Which one is the highest ranking predictive model?
6. (VERY HARD, Optional and without a coded answer) This is for people that want to explore beyond what we did with a simple linear regression. All of the models we used assumed normally distributed observations. The best fitting model is unable to explain the observed distribution of prices, and the reason for this is that prices are not normally distributed around a mean. Prices are chosen as multiple of a thousand, and usually they are grouped into tiers. It's not unusual to have multiple cars sold at exactly the same price. This leads to heteroskedastic observations across brands and car names, and the assumption of a unique observational noise parameter breaks down. We don't have a concrete answer on what observational distribution should really underlie the observations. We want to invite you to try to think:
    a. How could you have different observational noises while still assuming that the observations are normally distributed?
    b. What kind of observational distribution could you use to account for the groupings of multiple selling prices at the same value, and then some other selling prices that have more dispersion? As a hint, try to look into mixture models.

In [ ]:
import itertools
import os
import warnings

import arviz as az
import numpy as np
import pandas as pd
import pymc as pm
import seaborn as sns
from matplotlib import pyplot as plt
from pytensor import tensor as pt
from sklearn.preprocessing import StandardScaler

sample_kwargs = {
    "random_seed": 42,
    "chains": 4,
    "idata_kwargs": {"log_likelihood": True},
}

plt.style.use("arviz-darkgrid")

In [ ]:
df = pd.read_csv(os.path.join("..", "data", "reduced_car_data.csv"), header=0)
df.head()

In [ ]:
sns.pairplot(df, hue="brand");

### EDA

In [ ]:
df = pd.read_csv(os.path.join("..", "data", "reduced_car_data.csv"), header=0)
df

In [ ]:
with warnings.catch_warnings():
    # We ignore the warnings that seaborn is changing the layout of the plotting style
    warnings.filterwarnings("ignore")
    sns.pairplot(df)

In this data, we can clearly see two things.

1. Selling price is positive and concentrated at low values with potentially long tails
2. There seems to be a strong non linear relation between selling price and year

The strong relationship between selling price and year makes sense. It's very common to say that an old used car is worth less than a new used car. To make sense about the non-linearity let's have a look at the distribution of selling price.

People reason strangely about numbers and values. They tend to think about orders of magnitude instead of the actual number that is presented to determine if one thing is more valuable than another. Since consumers will have this at their core, it's also natural that sellers will set prices based on order of magnitude considerations more than on actual monetary value or costs of a car. If this were true, taking the logarithm of the selling price should lead to a distribution that is less skewed and to a clearer linear relationship between year and price.

In [ ]:
df2 = df[["year"]].assign(log_selling_price=np.log(df.selling_price))
with warnings.catch_warnings():
    # We ignore the warnings that seaborn is changing the layout of the plotting style
    warnings.filterwarnings("ignore")
    sns.pairplot(df2)

This looks much better now.
- The selling price is not completely normal, but it is less skewed than before
- The relationship between the log of selling price and year seems linear, albeit with a lot of dispersion (potentially due to other factors like km driven or engine CC)

Let's now look at the whole pairplot when we use the log of selling price. We will also standardize the numerical features. The reason we do this is that a linear regression where the features have non-zero mean, will affect the estimated intercept. In turn, this will make it harder to set a prior on the intercept when adding in new features to the regression. Furthermore, if the features are not rescaled, it would be necessary to set a different prior for each feature's slope parameter, to make the model assume that all features could be equally weighted a priori.

In [ ]:
scaler = StandardScaler()
features = ["year", "log_km_driven", "kmpl", "engine_cc"]
categoricals = ["name", "brand", "seats"]
categories = {k: pd.factorize(df[k], sort=True) for k in categoricals}
brands = list(categories["brand"][1])
name_to_brand_idx = {
    i: brands.index(n.split(" ")[0]) for i, n in enumerate(categories["name"][1])
}
observed = "selling_price"
df2 = pd.DataFrame(
    scaler.fit_transform(df[features]),
    columns=features,
)
df2 = df2.assign(
    log_selling_price=np.log(df[observed]),
    **{k: df[k] for k in categoricals},
)
df2

One other thing that we can say is that number of seats is a rather bad categorical feature. Almost all of the observations were taken for 5 seat vehicles, and there doesn't seem to be much structure for the other seats. We won't add it as a regressor via a slope, but we could consider it as an added intercept that depends on the number of seats. For now, we will simply ignore it.

In [ ]:
with warnings.catch_warnings():
    # We ignore the warnings that seaborn is changing the layout of the plotting style
    warnings.filterwarnings("ignore")
    sns.pairplot(df2, hue="seats")

Now let's have a look at how brand structures the data

In [ ]:
with warnings.catch_warnings():
    # We ignore the warnings that seaborn is changing the layout of the plotting style
    warnings.filterwarnings("ignore")
    sns.pairplot(df2, hue="brand");

It's a bit hard to disentangle what is going on here, but it seems that brand influences selling price a lot. It's not clear if it does so mediated by some of the other features in the dataset, if it directly offsets the selling price, or both. Let's simply focus on the distribution of selling price per brand.

In [ ]:
bins = np.linspace(df2.log_selling_price.min(), df2.log_selling_price.max(), 31)
colors = [
    plt.get_cmap("magma")(x) for x in np.linspace(0, 1, len(categories["brand"][1]))
]
nrows = np.ceil(len(categories["brand"][1]) / 4)
fig, axs = plt.subplots(int(nrows), 4, figsize=(7, nrows * 1.5), layout="constrained")
for brand, color, ax in zip(categories["brand"][1], colors, axs.flatten()):
    subdf = df2.query("brand == @brand")
    ax.hist(subdf.log_selling_price.values, bins=bins, density=True, color=color)
    ax.set_title(brand)

It looks like the brand influences the distribution of selling prices. It seems like the mean is shifted across brands and the dispersion is different across brands.

In [ ]:
bins = np.linspace(df2.log_selling_price.min(), df2.log_selling_price.max(), 31)
fig, axs = plt.subplots(
    len(categories["brand"][1]), 5, figsize=(14, 20), layout="constrained"
)
for axs_row, (brand, subdf) in zip(axs, df2.groupby("brand")):
    for ax, (name, by_name) in itertools.zip_longest(
        axs_row, subdf.groupby("name"), fillvalue=(None, None)
    ):
        if isinstance(ax, tuple):
            break
        if by_name is None:
            ax.set_axis_off()
        else:
            ax.hist(by_name.log_selling_price.values, bins=bins, density=True)
            ax.set_title(name)

Looking at individual car models, the story seems to be the same as for the overall brand.

### Linear models

The first model simply estimates the mean selling price and observational noise, and it is useful for setting priors.

In [ ]:
with pm.Model(coords={"obs_idx": df2.index}) as m0:
    intercept = pm.Normal("intercept", 6, 1)
    mu = intercept
    sigma = pm.Exponential("sigma", lam=1)
    pm.Normal("log_price", mu, sigma, observed=df2.log_selling_price, dims="obs_idx")
    idata = i0 = pm.sample(**sample_kwargs)
    idata.extend(pm.sample_prior_predictive())
    idata.extend(pm.sample_posterior_predictive(idata))
az.plot_trace(idata);

In [ ]:
az.plot_ppc(i0, group="prior", num_pp_samples=100);

In [ ]:
az.plot_ppc(i0, num_pp_samples=100);

In [ ]:
coords = {"feature": features, "obs_idx": df2.index}
with pm.Model(coords=coords) as m1:
    intercept = pm.Normal("intercept", 6, 1)
    slopes = pm.Normal("slopes", 0, 0.3, dims="feature")
    mu = intercept + pm.math.dot(df2[coords["feature"]].values, slopes)
    sigma = pm.Exponential("sigma", lam=1)
    pm.Normal("log_price", mu, sigma, observed=df2.log_selling_price, dims="obs_idx")
    idata = i1 = pm.sample(**sample_kwargs)
    idata.extend(pm.sample_prior_predictive())
    idata.extend(pm.sample_posterior_predictive(idata))

In [ ]:
az.plot_ppc(idata, group="prior", num_pp_samples=100);

In [ ]:
az.plot_trace(idata);

In [ ]:
az.plot_ppc(idata, num_pp_samples=100);

### Hierarchical brand models

Now, we will look into the hierarchical structure for brands. We will try to use a centered parametrization for the intercept variable.


In [ ]:
coords = {
    "feature": features,
    "brand": categories["brand"][1],
    "obs_idx": df2.index,
}
with pm.Model(coords=coords) as m2:
    idx = categories["brand"][0]
    intercept_global = pm.Normal("intercept_global", 6, 1)
    intercept_brand_scale = pm.HalfNormal("intercept_brand_scale", 0.5)
    intercept = pm.Normal(
        "intercept", intercept_global, intercept_brand_scale, dims="brand"
    )
    slopes = pm.Normal("slopes", 0, 0.1, dims="feature")
    mu = intercept[idx] + pm.math.dot(df2[coords["feature"]].values, slopes)
    sigma = pm.Exponential("sigma", lam=1)
    pm.Normal("log_price", mu, sigma, observed=df2.log_selling_price, dims="obs_idx")
    idata = i2 = pm.sample(**sample_kwargs)
    idata.extend(pm.sample_prior_predictive())
    idata.extend(pm.sample_posterior_predictive(idata))

In [ ]:
az.plot_ppc(idata, group="prior", num_pp_samples=100);

In [ ]:
az.plot_trace(idata);

In [ ]:
az.plot_ppc(idata, num_pp_samples=100);

Now we will use the hierarchical structure only on the slopes

In [ ]:
coords = {
    "feature": features,
    "brand": categories["brand"][1],
    "obs_idx": df2.index,
}
with pm.Model(coords=coords) as m3:
    idx = categories["brand"][0]
    intercept = pm.Normal("intercept", 6, 1, dims="brand")
    slopes_global = pm.Normal("slopes_global", 0, 0.1, dims="feature")
    slopes_brand_scale = pm.HalfNormal("slopes_brand_scale", 0.2)
    slopes_brand = pm.Normal("slopes_brand", 0, 0.1, dims=("feature", "brand"))
    slopes = slopes_global[:, None] + slopes_brand_scale * slopes_brand
    mu = intercept[idx] + sum(
        [
            df2[feature].values * slopes[i, idx]
            for i, feature in enumerate(coords["feature"])
        ],
    )
    sigma = pm.Exponential("sigma", lam=1)
    pm.Normal("log_price", mu, sigma, observed=df2.log_selling_price, dims="obs_idx")
    idata = i3 = pm.sample(**sample_kwargs)
    idata.extend(pm.sample_posterior_predictive(idata))
az.plot_trace(idata);

In [ ]:
az.plot_ppc(idata, num_pp_samples=100);

And finally have the hierarchical structure on both

In [ ]:
coords = {
    "feature": features,
    "brand": categories["brand"][1],
    "obs_idx": df2.index,
}
with pm.Model(coords=coords) as m4:
    idx = categories["brand"][0]
    intercept_global = pm.Normal("intercept_global", 6, 1)
    intercept_brand_scale = pm.HalfNormal("intercept_brand_scale", 0.5)
    intercept = pm.Normal(
        "intercept", intercept_global, intercept_brand_scale, dims="brand"
    )
    slopes_global = pm.Normal("slopes_global", 0, 0.1, dims="feature")
    slopes_brand_scale = pm.HalfNormal("slopes_brand_scale", 0.2)
    slopes_brand = pm.Normal("slopes_brand", 0, 0.1, dims=("feature", "brand"))
    slopes = slopes_global[:, None] + slopes_brand_scale * slopes_brand
    mu = intercept[idx] + sum(
        [
            df2[feature].values * slopes[i, idx]
            for i, feature in enumerate(coords["feature"])
        ],
    )
    sigma = pm.Exponential("sigma", lam=1)
    pm.Normal("log_price", mu, sigma, observed=df2.log_selling_price, dims="obs_idx")
    idata = i4 = pm.sample(**sample_kwargs)
    idata.extend(pm.sample_posterior_predictive(idata))
az.plot_trace(idata);

In [ ]:
az.plot_ppc(idata, num_pp_samples=100);

### Nested hierarchical models

To begin with, we will use the nested hierarchy for the intercept parameter and not include any hierarchical structure for the slopes

In [ ]:
coords = {
    "feature": features,
    "brand": categories["brand"][1],
    "name": categories["name"][1],
    "obs_idx": df2.index,
}
with pm.Model(coords=coords) as m5:
    idx_brand = categories["brand"][0]
    idx_name = categories["name"][0]
    intercept_global = pm.Normal("intercept_global", 6, 1)
    intercept_brand = pm.Normal("intercept_brand", 0, 1, dims="brand")
    intercept_brand_scale = pm.HalfNormal("intercept_brand_scale")
    intercept_name_scale = pm.HalfNormal("intercept_name_scale")
    intercept_name = pm.Normal(
        "intercept_name",
        (intercept_global + intercept_brand_scale * intercept_brand)[
            np.array(list(name_to_brand_idx.values()))
        ],
        intercept_name_scale,
        dims="name",
    )
    slopes = pm.Normal("slopes", 0, 0.1, dims="feature")
    mu = intercept_name[idx_name] + pm.math.dot(df2[coords["feature"]].values, slopes)
    sigma = pm.Exponential("sigma", lam=1)
    pm.Normal("log_price", mu, sigma, observed=df2.log_selling_price, dims="obs_idx")
    idata = i5 = pm.sample(**sample_kwargs)
    idata.extend(pm.sample_posterior_predictive(idata))
az.plot_trace(idata);

In [ ]:
az.plot_ppc(idata, num_pp_samples=100);

Now let's add the brand hierarchy of the slopes again for our full model

In [ ]:
coords = {
    "feature": features,
    "brand": categories["brand"][1],
    "name": categories["name"][1],
    "obs_idx": df2.index,
}
with pm.Model(coords=coords) as m6:
    idx_brand = categories["brand"][0]
    idx_name = categories["name"][0]
    intercept_global = pm.Normal("intercept_global", 6, 1)
    intercept_brand = pm.Normal("intercept_brand", 0, 1, dims="brand")
    intercept_brand_scale = pm.HalfNormal("intercept_brand_scale")
    intercept_name_scale = pm.HalfNormal("intercept_name_scale")
    intercept_name = pm.Normal(
        "intercept_name",
        (intercept_global + intercept_brand_scale * intercept_brand)[
            np.array(list(name_to_brand_idx.values()))
        ],
        intercept_name_scale,
        dims="name",
    )
    slopes_global = pm.Normal("slopes_global", 0, 0.1, dims="feature")
    slopes_brand_scale = pm.HalfNormal("slopes_brand_scale", 0.2)
    slopes_brand = pm.Normal("slopes_brand", 0, 0.1, dims=("feature", "brand"))
    slopes = slopes_global[:, None] + slopes_brand_scale * slopes_brand
    mu = intercept_name[idx_name] + sum(
        [
            df2[feature].values * slopes[i, idx]
            for i, feature in enumerate(coords["feature"])
        ],
    )
    sigma = pm.Exponential("sigma", lam=1)
    pm.Normal("log_price", mu, sigma, observed=df2.log_selling_price, dims="obs_idx")
    idata = i6 = pm.sample(**sample_kwargs)
    idata.extend(pm.sample_posterior_predictive(idata))
az.plot_trace(idata);

In [ ]:
az.plot_ppc(idata, num_pp_samples=100);

In the next section we will see that this is our best predictive model. Despite that, the model isn't great. To see that we will have to look at the per brand selling price.

In [ ]:
nrows = np.ceil(len(categories["brand"][1]) / 4)
fig, axs = plt.subplots(int(nrows), 4, figsize=(12, 10))
for i, (ax, brand) in enumerate(zip(axs.flatten(), categories["brand"][1])):
    az.plot_ppc(
        idata,
        num_pp_samples=100,
        coords={"obs_idx": df2.query(f"brand == @brand").index},
        ax=ax,
    )
    ax.set_title(brand)
    if i // 4 < nrows - 1:
        ax.set_xlabel("")

If you look at these posterior predictives per brand, you will notice that the brand observations do not seem normal. At most, they look like mixtures of normals, some of which have very small variance. Let's zoom in even more and look at the posterior predictive per car name inside each brand.

In [ ]:
fig, axs = plt.subplots(
    len(categories["brand"][1]), 5, figsize=(14, 25), layout="constrained"
)
for axs_row, (brand, subdf) in zip(axs, df2.groupby("brand")):
    for ax, (name, by_name) in itertools.zip_longest(
        axs_row, subdf.groupby("name"), fillvalue=(None, None)
    ):
        if isinstance(ax, tuple):
            break
        if by_name is None:
            ax.set_axis_off()
        else:
            az.plot_ppc(
                idata,
                num_pp_samples=100,
                coords={"obs_idx": by_name.index},
                ax=ax,
                legend=False,
            )
            ax.set(title=name, xlabel="")

Here we have the key issue with all of our models so far, the observations are clearly not normally distributed. For example, TATA MANZA is made up by two price values, some of which have more occurences than the other. This is also the case for some cars from Ford. This means that our assumed observational distribution is wrong to begin with. There is two last things that we could do to improve it while still assuming normally distributed observations around some means:
1. Make the name scale parameter change by brand. This means that the expected price of each car model from the same brand will be pulled towards the brand expected price with different strengths
2. Make the observational noise, `sigma` be brand or name dependent. This would allow the model predictions to have different spreads per brand or car model. Implementing this in practice would not be easy given that the observations are still not normally distributed and some car models have very few data points.

The real way to resolve this fundamental problem with the model is to think deeply about how to change the observational distribution and move away from a simple Gaussian. It might make sense to consider mixture models with some forms of special parametrization. We wont touch upon those though.

### Model comparison

We can use arviz to run a leave one out kind of cross validation estimate. For these models, we will get warnings regarding the shape of a Pareto distribution. This warning comes from a diagnostic statistic called $\hat{k}$. It essentially measures the influence of the individual observations on the goodness of the estimate of LOO predictive accuracy. If we have some individual observations that are extremely influencial for the fit results, this implies that the model predictions might be very different when said observations are left out or not. In turn, this makes the approximation method used by `arviz` to be potentially biased, so a better way to approach the computation of LOO would be to actually fit the model on a subset of data without the influencial point and then compute the actual LOO score, instead of relying on the Pareto approximation that `arviz` uses by default.

We won't do that here, but we will show you how to visualize the $\hat{k}$ and see the number of problematic observations.

In [ ]:
cmp = az.compare(
    {
        "Single intercept": i0,
        "Simple linear": i1,
        "Brand Hier intercept": i2,
        "Brand Hier slope": i3,
        "Brand Hier intercept and brand Hier slope": i4,
        "Brand/name Hier intercept": i5,
        "Brand/name Hier intercept and brand Hier slope": i6,
    },
    ic="loo",
)
cmp

In [ ]:
plt.figure(figsize=(12, 6), layout="tight")
az.plot_compare(cmp);

In [ ]:
loo = az.loo(i2, pointwise=True)

az.plot_khat(loo, show_bins=True);

In [ ]:
loo = az.loo(i3, pointwise=True)
print(loo)

az.plot_khat(loo, show_bins=True);

In [ ]:
loo = az.loo(i4, pointwise=True)
print(loo)

az.plot_khat(loo, show_bins=True);

In [ ]:
loo = az.loo(i5, pointwise=True)
print(loo)

az.plot_khat(loo, show_bins=True);

In [ ]:
loo = az.loo(i6, pointwise=True)
print(loo)

az.plot_khat(loo, show_bins=True);